# Compilation Strategies in TensorLogic

TensorLogic supports multiple compilation strategies that implement different semantic interpretations of logical operations. This is crucial for handling uncertainty and enabling gradient-based training.

## What You'll Learn

1. The five built-in compilation strategies
2. How each strategy computes AND, OR, NOT
3. When to use each strategy
4. The "Train Soft, Infer Hard" pattern

In [ ]:
from tensorlogic import (
    create_backend,
    create_strategy,
    get_available_strategies,
)
import numpy as np

backend = create_backend()
print(f"Backend: {type(backend).__name__}")
print(f"\nAvailable strategies: {get_available_strategies()}")

## The Five Compilation Strategies

| Strategy | AND | OR | Use Case |
|----------|-----|-----|----------|
| **soft_differentiable** | a × b | a + b - ab | Training neural predicates |
| **hard_boolean** | step(a × b) | step(max(a,b)) | Production inference |
| **godel** | min(a, b) | max(a, b) | Conservative fuzzy reasoning |
| **product** | a × b | a + b - ab | Independent probability modeling |
| **lukasiewicz** | max(0, a+b-1) | min(1, a+b) | Bounded arithmetic |

In [ ]:
# Create test data with uncertain predicates (values in [0, 1])
p = np.array([0.8, 0.6, 0.4, 0.2])
q = np.array([0.9, 0.5, 0.3, 0.1])

print("Test Data (Uncertain Predicates):")
print(f"  P = {p}")
print(f"  Q = {q}")

## Comparing AND Operations

In [ ]:
print("\nAND Operation Comparison (P AND Q):")
print("=" * 70)

strategies_and = [
    ("soft_differentiable", "a × b", "Gradients flow through"),
    ("hard_boolean", "step(a × b)", "Binary output"),
    ("godel", "min(a, b)", "Conservative: takes worst case"),
    ("product", "a × b", "Same as soft_differentiable"),
    ("lukasiewicz", "max(0, a+b-1)", "Bounded: stricter requirement"),
]

for name, formula, description in strategies_and:
    strategy = create_strategy(name, backend=backend)
    result = strategy.compile_and(p, q)
    result_arr = np.asarray(result)
    print(f"\n{name}:")
    print(f"  Formula: {formula}")
    print(f"  Result:  {np.array2string(result_arr, precision=3)}")
    print(f"  Note:    {description}")

### Understanding the Results

For the first element where P=0.8, Q=0.9:

- **soft_differentiable/product**: 0.8 × 0.9 = 0.72
- **hard_boolean**: step(0.72) = 1.0 (thresholded)
- **godel**: min(0.8, 0.9) = 0.8
- **lukasiewicz**: max(0, 0.8 + 0.9 - 1) = 0.7

## Comparing OR Operations

In [ ]:
print("\nOR Operation Comparison (P OR Q):")
print("=" * 70)

strategies_or = [
    ("soft_differentiable", "a + b - ab", "Probabilistic union"),
    ("hard_boolean", "step(max(a,b))", "Binary output"),
    ("godel", "max(a, b)", "Takes best case"),
    ("product", "a + b - ab", "Same as soft_differentiable"),
    ("lukasiewicz", "min(1, a+b)", "Bounded: sum capped at 1"),
]

for name, formula, description in strategies_or:
    strategy = create_strategy(name, backend=backend)
    result = strategy.compile_or(p, q)
    result_arr = np.asarray(result)
    print(f"\n{name}:")
    print(f"  Formula: {formula}")
    print(f"  Result:  {np.array2string(result_arr, precision=3)}")
    print(f"  Note:    {description}")

## Comparing NOT Operations

In [ ]:
print("\nNOT Operation Comparison (NOT P):")
print("=" * 70)

strategies_not = [
    ("soft_differentiable", "1 - a"),
    ("hard_boolean", "1 - step(a)"),
    ("godel", "1 - a"),
    ("product", "1 - a"),
    ("lukasiewicz", "1 - a"),
]

for name, formula in strategies_not:
    strategy = create_strategy(name, backend=backend)
    result = strategy.compile_not(p)
    result_arr = np.asarray(result)
    print(f"\n{name}:")
    print(f"  Formula: {formula}")
    print(f"  Result:  {np.array2string(result_arr, precision=3)}")

## Real-World Example: Evidence Combination

Consider combining DNA evidence with witness testimony in a legal case.

In [ ]:
# Evidence confidence levels
dna_evidence = 0.85       # 85% confident from DNA
witness_testimony = 0.70  # 70% confident from witness

print("Legal Case: Combining Evidence")
print("=" * 60)
print(f"DNA Evidence:      {dna_evidence:.0%} confidence")
print(f"Witness Testimony: {witness_testimony:.0%} confidence")

print("\n--- Scenario 1: Both evidences must hold (AND) ---")
for strategy_name in ["soft_differentiable", "godel", "lukasiewicz"]:
    strategy = create_strategy(strategy_name, backend=backend)
    result = strategy.compile_and(
        np.array([dna_evidence]),
        np.array([witness_testimony])
    )
    print(f"{strategy_name:>20}: {float(result[0]):.3f}")

print("\n--- Scenario 2: Either evidence suffices (OR) ---")
for strategy_name in ["soft_differentiable", "godel", "lukasiewicz"]:
    strategy = create_strategy(strategy_name, backend=backend)
    result = strategy.compile_or(
        np.array([dna_evidence]),
        np.array([witness_testimony])
    )
    print(f"{strategy_name:>20}: {float(result[0]):.3f}")

### Analysis

**For AND (both required)**:
- **soft_differentiable**: 0.85 × 0.70 = 0.595 (probabilistic)
- **godel**: min(0.85, 0.70) = 0.70 (conservative)
- **lukasiewicz**: max(0, 0.85 + 0.70 - 1) = 0.55 (strictest)

**For OR (either suffices)**:
- **soft_differentiable**: 0.85 + 0.70 - 0.595 = 0.955
- **godel**: max(0.85, 0.70) = 0.85
- **lukasiewicz**: min(1, 0.85 + 0.70) = 1.0

## The "Train Soft, Infer Hard" Pattern

A key workflow in neural-symbolic systems:

1. **Training Phase**: Use `soft_differentiable` strategy
   - Gradients flow through product operations
   - Neural predicates can learn from logical constraints

2. **Inference Phase**: Use `hard_boolean` strategy
   - Exact 0/1 outputs, no ambiguity
   - No hallucinations from soft interpolation

In [ ]:
# Simulating the Train Soft, Infer Hard pattern

# Uncertain predictions from a neural model (values in [0, 1])
neural_predictions = np.array([0.92, 0.78, 0.45, 0.31, 0.08])
labels = ["Entity A", "Entity B", "Entity C", "Entity D", "Entity E"]

print("Neural Model Predictions:")
print("=" * 50)
for label, pred in zip(labels, neural_predictions):
    print(f"  {label}: {pred:.2f}")

# During training: use soft values for gradient computation
soft_strategy = create_strategy("soft_differentiable", backend=backend)

# During inference: threshold to hard boolean
hard_strategy = create_strategy("hard_boolean", backend=backend)

print("\nTraining (soft_differentiable):")
print(f"  Values: {neural_predictions}")
print("  (Gradients can flow through these soft values)")

# For hard strategy, we apply step function via NOT(NOT(x)) to threshold
hard_result = (neural_predictions > 0.5).astype(np.float32)
print("\nInference (hard_boolean, threshold=0.5):")
print(f"  Values: {hard_result}")
print("  (Binary decisions: 0 or 1)")

print("\nFinal Predictions:")
for label, soft, hard in zip(labels, neural_predictions, hard_result):
    decision = "Yes" if hard > 0 else "No"
    print(f"  {label}: {soft:.2f} -> {decision}")

## Strategy Selection Guide

| Use Case | Recommended Strategy | Why |
|----------|---------------------|-----|
| Training neural networks | `soft_differentiable` | Gradients flow through |
| Production inference | `hard_boolean` | Exact, interpretable results |
| Medical diagnosis | `godel` | Conservative (worst-case) |
| Independent events | `product` | Probabilistic semantics |
| Resource allocation | `lukasiewicz` | Bounded, strict requirements |

In [ ]:
# Example: Medical Diagnosis with Gödel Logic
# (Conservative approach - if ANY symptom is weak, the overall diagnosis confidence drops)

symptom1 = 0.9  # High confidence for symptom 1
symptom2 = 0.5  # Medium confidence for symptom 2
symptom3 = 0.3  # Low confidence for symptom 3

symptoms = np.array([symptom1, symptom2, symptom3])

print("Medical Diagnosis Example")
print("=" * 50)
print(f"Symptom 1: {symptom1:.0%} confidence")
print(f"Symptom 2: {symptom2:.0%} confidence")
print(f"Symptom 3: {symptom3:.0%} confidence")

print("\nDiagnosis requires ALL three symptoms (AND):")

for strategy_name in ["godel", "product", "lukasiewicz"]:
    strategy = create_strategy(strategy_name, backend=backend)
    
    # Chain AND operations: symptom1 AND symptom2 AND symptom3
    result = strategy.compile_and(
        np.array([symptom1]),
        np.array([symptom2])
    )
    result = strategy.compile_and(result, np.array([symptom3]))
    
    confidence = float(result[0])
    print(f"\n{strategy_name}:")
    print(f"  Overall confidence: {confidence:.1%}")
    
    if strategy_name == "godel":
        print(f"  (min of all: {min(symptom1, symptom2, symptom3):.0%})")
    elif strategy_name == "product":
        print(f"  (product: {symptom1 * symptom2 * symptom3:.1%})")
    elif strategy_name == "lukasiewicz":
        luk = max(0, symptom1 + symptom2 - 1)
        luk = max(0, luk + symptom3 - 1)
        print(f"  (bounded sum: {luk:.1%})")

## Summary

**Key Takeaways**:

1. **soft_differentiable**: Best for training; gradients flow through operations
2. **hard_boolean**: Best for production; gives exact binary outputs
3. **godel**: Conservative; min/max gives worst/best case
4. **product**: Probabilistic; treats inputs as independent probabilities
5. **lukasiewicz**: Strictest; bounded arithmetic, good for resource constraints

**Best Practice**: Train with `soft_differentiable`, deploy with `hard_boolean`

## Next Steps

- **04_temperature_control.ipynb**: Explore deductive vs analogical reasoning